## Day 19: Beacon Scanner

https://adventofcode.com/2021/day/19

For a moment, consider only two dimensions. Suppose you have the following scanner reports: `day19-1.txt`

For example, here is an arrangement of beacons as seen from a scanner in the same position but in different orientations: `day19-2.txt`

By finding pairs of scanners that both see at least 12 of the same beacons, you can assemble the entire map. For example, consider the following report: `day19-3.txt` - In total, there are 79 beacons.

Assemble the full map of beacons. How many beacons are there?

In [35]:
import numpy as np
import pandas as pd
import itertools


def parse_report(filename):
    """
    Parse raw scanner report into useful format

    """

    with open(filename, "r") as infile:
        raw_data = [l.strip() for l in infile]

    raw_data.append("")

    # scanner count, only used for example 2
    scanner = 0
    processed_data = {}
    for i in range(0, len(raw_data)):
        if "scanner" in raw_data[i]:
            scanner_label = raw_data[i].replace("---", "").strip()
            scanner_data = []
        elif raw_data[i] == "":
            if filename == "day19-2.txt":
                scanner_label += ["a", "b", "c", "d", "e"][scanner]
                print(scanner_label)
                scanner += 1

            # Convert to numpy matrix
            processed_data[scanner_label] = np.matrix(scanner_data)
        else:
            # add z axis to first example for consistency
            line_data = list(map(int, raw_data[i].split(",")))
            if len(line_data) < 3:
                line_data.append(0)
            scanner_data.append(line_data)

    return processed_data

def rotate_report(scanner_report):
    """
    Rotate scanner report, return list with all 24 orientations
    
    """
    
    # Permutations of the columns and positive/negative
    rotated_reports = []
    for l in itertools.product([1, -1], repeat = 3):
        for p in itertools.permutations([0, 1, 2]):
            r = scanner_report[:, list(p)]
            for j in range(0, 3):
                r[:, j] *= l[j]
            # Only add if we don't already have this or its opposite
            if not any([np.array_equal(x, r) or np.array_equal(-x, r) for x in rotated_reports]):
                rotated_reports.append(r)

    if len(rotated_reports) != 24:
        print(F"Unexpected number of rotated reports: {len(rotated_reports)} (expected 24)")
        
    return rotated_reports
    
def calculate_report_deltas(scanner_report):
    """
    Calculates differences between pairs of coordinates in a scanner report
    
    """
    
    diffs = []
    
    for i in scanner_report:
        for j in scanner_report:
            diffs.append((i-j).tolist()[0])
    
    return diffs

def align_scanners(scanner_data):
    """
    Align scanners and find beacons in overlapping detection cubes
    
    """
    
    # For any pair of scanners, determine if they have an overlapping 
    # detection cube and return the beacons in the intersection of the cubes
    #
    # Given a pair of scanners A and B,
    #   - Determine if there is a rotation of scanner B relative to scanner A
    #     that results in the existence of common beacons (assume this happens when,
    #     for a particular rotation of scanner B, at least 12 of scanner B
    #     reports can be obtained by shifting scanner B report from scanner A report 
    #     in scanner A's x, y and z direction)
    #
    # PSEUDOCODE -
    # for r in rotations:
    #   report_B_rotated = rotate(report_B by r)
    #   if at lea12 identical entries in report_B_rotated - report_A:
    #     A and B are overlapping
    #     list beacons in A frame of reference
    #     list beacons in B frame of reference
    #     list coordinates of B in A frame of reference
    #
    # IF
    #   A and B have overlapping detection cubes
    # THEN
    #   the coordinates for the beacons in the overlap are related by a linear transformation
    #   consisting of a rotation and a shift M*x + t; M is the rotation matrix and t is the shift
    
    for s in list(itertools.combinations(scanner_data.keys(), 2)):
        print(s)
        sA = scanner_data[s[0]].copy()
        sA_diffs = calculate_report_deltas(sA)
        sB = scanner_data[s[1]].copy()
        sB_rot = rotate_report(sB)
        found_match = False
        while not found_match and len(sB_rot) > 0:
            r = sB_rot.pop()
            r_diffs = calculate_report_deltas(r)
            common_distances = [x for x in sA_diffs if x in r_diffs and x != [0,0,0]]
            if len(common_distances) > 100:
                print(F"Found a match: {len(common_distances)} common distances")
                found_match = True
                
        
    return 0
    


p = parse_report("day19-3.txt")
align_scanners(p);


('scanner 0', 'scanner 1')
Found a match: 132 common distances
('scanner 0', 'scanner 2')
('scanner 0', 'scanner 3')
('scanner 0', 'scanner 4')
('scanner 1', 'scanner 2')
('scanner 1', 'scanner 3')
Found a match: 132 common distances
('scanner 1', 'scanner 4')
Found a match: 132 common distances
('scanner 2', 'scanner 3')
('scanner 2', 'scanner 4')
Found a match: 132 common distances
('scanner 3', 'scanner 4')
